# 🏗️ FUNDAÇÃO E PREPARAÇÃO

Carrega configurações do ambiente e prepara diretórios.

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Carregar .env
env_file = f".env.{os.getenv('ENVIRONMENT', 'development')}"
if Path(f"../{env_file}").exists():
    load_dotenv(f"../{env_file}")
elif Path("../.env").exists():
    load_dotenv("../.env")

# Criar diretórios
Path("pipeline_data/documents").mkdir(parents=True, exist_ok=True)
Path("pipeline_data/processed").mkdir(parents=True, exist_ok=True)
Path("pipeline_data/chunks").mkdir(parents=True, exist_ok=True)
Path("pipeline_data/embeddings").mkdir(parents=True, exist_ok=True)

print("✅ Configuração carregada e diretórios criados")

✅ Configuração carregada e diretórios criados
